In [1]:
!pip install xgboost
!pip install seaborn
!pip install pandas scikit-learn

In [2]:
import pandas as pd
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from collections import defaultdict
from pathlib import Path
import os
import itertools 
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import roc_auc_score, make_scorer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
seed =1380
np.random.seed(seed)
random.seed(seed) 
os.environ['PYTHONHASHSEED']=str(seed)

## **1. EF. fMRI filtered for 75 regions**

In [4]:
fmri75= pd.read_csv('fmri_without_duplicates.csv')
fmri75.drop_duplicates()
fmri75

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,0.664985,0.299166,0.073119,0.407197,0.240695,0.669216,0.168402,0.111856,0.097451,0.082961,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [5]:
target =fmri75.target
data =fmri75.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.702 ± 0.118
Random Forest: Mean score using nested cross-validation is: 0.658 ± 0.114
SVC: Mean score using nested cross-validation is: 0.654 ± 0.117
XGBoost: Mean score using nested cross-validation is: 0.750 ± 0.111


## 2. **EF. Lipids 75 regions**

In [6]:
lipids75= pd.read_csv('75reg_without_duplicates.csv')
lipids75.drop_duplicates()
lipids75

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [7]:
target =lipids75.target
data =lipids75.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.619 ± 0.083
Random Forest: Mean score using nested cross-validation is: 0.625 ± 0.135
SVC: Mean score using nested cross-validation is: 0.599 ± 0.105
XGBoost: Mean score using nested cross-validation is: 0.620 ± 0.130


## **3. Morphometry 123 subjects**

In [8]:
morph75= pd.read_csv('morphometry_no_duplicates.csv')
morph75.drop_duplicates()
morph75

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,superiorfrontal_CurvInd_lh,superiorparietal_CurvInd_lh,superiortemporal_CurvInd_lh,supramarginal_CurvInd_lh,frontalpole_CurvInd_lh,temporalpole_CurvInd_lh,transversetemporal_CurvInd_lh,insula_CurvInd_lh,ID,target
0,18693,274,14134,50431,7221,3499,4535,1996,792,1462,...,11.3,7.7,4.6,6.8,0.8,1.3,0.4,3.7,A00007409,1
1,2489,181,12436,50238,7236,3330,4463,2107,546,1044,...,13.1,6.7,6.0,6.2,1.0,1.2,1.2,7.9,A00022500,0
2,7704,231,13415,54324,8037,2523,4709,2499,1032,1662,...,15.4,6.7,7.1,6.9,1.7,1.2,0.8,4.9,A00024953,0
3,10600,304,13585,62141,7035,3302,5281,2180,854,1831,...,11.9,7.6,4.5,5.1,0.8,0.7,1.4,3.5,A00001251,0
4,10407,209,24048,59909,8349,2858,4452,2191,1066,1759,...,10.5,7.0,6.1,7.5,0.8,0.8,1.0,4.5,A00037495,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,5177,187,17224,58244,7490,3822,4449,2083,857,2315,...,9.2,6.5,4.2,5.6,1.2,0.9,0.5,5.0,A00022727,1
119,4339,148,12149,47024,7194,3371,4094,1854,496,1055,...,10.5,6.7,4.9,7.2,1.1,1.1,0.6,4.2,A00022653,1
120,10667,193,14434,55105,7806,4303,5323,2368,636,1077,...,18.9,11.3,9.3,7.7,0.8,1.0,0.9,5.9,A00010150,1
121,5873,242,17772,52836,7745,3865,5033,2401,703,1115,...,10.5,7.7,4.2,5.7,0.8,0.8,0.7,5.2,A00016723,0


In [9]:
target =morph75.target
data =morph75.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.533 ± 0.160
Random Forest: Mean score using nested cross-validation is: 0.545 ± 0.114
SVC: Mean score using nested cross-validation is: 0.624 ± 0.124
XGBoost: Mean score using nested cross-validation is: 0.527 ± 0.117


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

## **4.1. EF. Morphometry+fMRI 123 subjects**

In [47]:
fmriandmorph= pd.read_csv('fmri_and_morp_123.csv')
fmriandmorph.drop_duplicates()
fmriandmorph

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,18693,274,14134,50431,7221,3499,4535,1996,792,1462,...,0.009590,0.172087,0.269736,0.165370,0.231563,0.136910,0.292152,0.399644,0,A00007409
1,2489,181,12436,50238,7236,3330,4463,2107,546,1044,...,0.191281,0.215860,0.195709,0.140152,0.267767,0.253835,0.444850,0.429113,1,A00022500
2,7704,231,13415,54324,8037,2523,4709,2499,1032,1662,...,-0.075162,-0.159331,-0.014370,-0.116377,-0.036073,0.292674,0.350473,0.483932,1,A00024953
3,10600,304,13585,62141,7035,3302,5281,2180,854,1831,...,0.103264,0.416603,0.083171,-0.177938,-0.020037,0.221657,0.559467,0.414580,1,A00001251
4,10407,209,24048,59909,8349,2858,4452,2191,1066,1759,...,-0.358174,0.548469,0.329123,0.301622,0.417169,0.293198,0.532769,0.347501,0,A00037495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,5177,187,17224,58244,7490,3822,4449,2083,857,2315,...,-0.022749,0.241897,0.182758,0.083769,0.155714,0.181779,0.366814,0.282371,0,A00022727
119,4339,148,12149,47024,7194,3371,4094,1854,496,1055,...,-0.246840,0.274223,0.202504,-0.238776,0.290823,0.209776,0.312112,0.302832,0,A00022653
120,10667,193,14434,55105,7806,4303,5323,2368,636,1077,...,-0.309410,0.015058,0.503868,0.520389,0.573108,0.567371,0.587601,0.686093,0,A00010150
121,5873,242,17772,52836,7745,3865,5033,2401,703,1115,...,-0.421263,0.216793,0.231175,-0.010506,0.010442,0.207897,0.454866,0.598393,1,A00016723


In [17]:
target =fmriandmorph.target
data =fmriandmorph.drop(columns=['target', 'ID'])

# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.663 ± 0.157
Random Forest: Mean score using nested cross-validation is: 0.647 ± 0.146
SVC: Mean score using nested cross-validation is: 0.583 ± 0.186
XGBoost: Mean score using nested cross-validation is: 0.654 ± 0.108


## **4.2 LF. Morphometry+fMRI 123 subjects. Logistic Regression**

In [32]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self
    
# fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,929) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(929,7599) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')

target =fmriandmorph.target
data_late =fmriandmorph.drop(columns=['target', 'ID'])

param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.700 ± 0.120
The mean score using nested cross-validation is: 0.708 ± 0.119
The mean score using nested cross-validation is: 0.707 ± 0.121


## **4.3 LF. Morphometry+fMRI 123 subjects. Random Forest**

In [34]:
# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipelines
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,929) )),
    ( "scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(929,7599) )),
    ( "scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])



# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

target =fmriandmorph.target
data_late =fmriandmorph.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    }

]


# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.663 ± 0.175
The mean score using nested cross-validation is: 0.691 ± 0.161
The mean score using nested cross-validation is: 0.670 ± 0.166


## **4.3 LF. Morphometry+fMRI 123 subjects. SVC**

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import VotingClassifier
import numpy as np

# Define your custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define three pipelines, each with an SVC classifier
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,929))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(929,7599))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])


# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

# Load your data and target
target =fmriandmorph.target
data_late =fmriandmorph.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.671 ± 0.132
The mean score using nested cross-validation is: 0.692 ± 0.125
The mean score using nested cross-validation is: 0.679 ± 0.128


## **4.4 LF. Morphometry+fMRI 123 subjects. XGBoost**

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target =fmriandmorph.target
data_late =fmriandmorph.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,929))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(929,7599))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.561 ± 0.177
The mean score using nested cross-validation is: 0.576 ± 0.175
The mean score using nested cross-validation is: 0.568 ± 0.171


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

## **5. EF. 75reg+fMRI 123 subjects**

In [42]:
fmri_with_75_reg = pd.read_csv('fmri_and_75_reg.csv')
fmri_with_75_reg.drop_duplicates()
fmri_with_75_reg

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [19]:
target =fmri_with_75_reg.target
data =fmri_with_75_reg.drop(columns=['target', 'ID'])

# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.710 ± 0.088
Random Forest: Mean score using nested cross-validation is: 0.722 ± 0.113
SVC: Mean score using nested cross-validation is: 0.723 ± 0.109
XGBoost: Mean score using nested cross-validation is: 0.728 ± 0.093


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

## **5.1 LF. 75reg+fMRI 123 subjects. Logistic regression**

In [43]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self
    
# fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,75) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(75,6745) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')

target =fmri_with_75_reg.target
data_late =fmri_with_75_reg.drop(columns=['target', 'ID'])

param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.729 ± 0.075
The mean score using nested cross-validation is: 0.738 ± 0.076
The mean score using nested cross-validation is: 0.738 ± 0.070


## **5.2 LF. 75reg+fMRI 123 subjects. Random Forest**

In [44]:
# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipelines
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,75) )),
    ( "scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(75,6745) )),
    ( "scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])



# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

target =fmri_with_75_reg.target
data_late =fmri_with_75_reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    }

]


# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.719 ± 0.102
The mean score using nested cross-validation is: 0.739 ± 0.080
The mean score using nested cross-validation is: 0.730 ± 0.088


## **5.3 LF. 75reg+fMRI 123 subjects. SVC**

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import VotingClassifier
import numpy as np

# Define your custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define three pipelines, each with an SVC classifier
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,75))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(75,6745))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])


# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

# Load your data and target
target =fmri_with_75_reg.target
data_late =fmri_with_75_reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.646 ± 0.100
The mean score using nested cross-validation is: 0.674 ± 0.084
The mean score using nested cross-validation is: 0.659 ± 0.095


## **5.4 LF. 75reg+fMRI 123 subjects. XGBoost**

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load your data and target
target =fmri_with_75_reg.target
data_late =fmri_with_75_reg.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,75))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(929,7599))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.620 ± 0.130
The mean score using nested cross-validation is: 0.637 ± 0.121
The mean score using nested cross-validation is: 0.626 ± 0.123


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

## **6. EF. 75reg+morph+fMRI 123 subjects**

In [4]:
fmri_and_morph_and_75reg = pd.read_csv('fmri_and_morp_and_75reg_123.csv')
fmri_and_morph_and_75reg

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [52]:
target =fmri_and_morph_and_75reg.target
data =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Logistic Regression: Mean score using nested cross-validation is: 0.709 ± 0.084
Random Forest: Mean score using nested cross-validation is: 0.718 ± 0.122
SVC: Mean score using nested cross-validation is: 0.692 ± 0.124


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

XGBoost: Mean score using nested cross-validation is: 0.718 ± 0.074


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

## **6.1 LF. 75reg+morph+fMRI 123 subjects. Logistic Regression**

In [5]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self
    
       # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,75) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(75,1002) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(1002,7672) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2), ('df3-clf3', pipe3)], voting='soft')

target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    },
        {
        'df3-clf3__clf__penalty': ['l1', 'l2'],
        'df3-clf3__clf__C': [0.1, 1, 10],
        'df3-clf3__clf__fit_intercept': [True, False],
        'df3-clf3__clf__solver': ['liblinear'],
        'df3-clf3__clf__random_state': [1380]
    }
    
]

from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.727 ± 0.080
The mean score using nested cross-validation is: 0.747 ± 0.080
The mean score using nested cross-validation is: 0.738 ± 0.088


## **6.2 LF. 75reg+morph+fMRI 123 subjects. Random Forest**

In [6]:
# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipelines
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,75) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1380))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(75,1002) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1380))
    ])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(1002,7672) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1380))
    ])


# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2),
    ('df3-clf3', pipe3)
    
], voting='soft')

# Load the data and target
target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    },
    {
        'df3-clf3__clf__n_estimators': [10, 50, 100],
        'df3-clf3__clf__max_depth': [3, 5, 7, 9],
        'df3-clf3__clf__random_state': [1380]
    }
    

]


# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.681 ± 0.121
The mean score using nested cross-validation is: 0.707 ± 0.103
The mean score using nested cross-validation is: 0.689 ± 0.111


## **6.3 LF. 75reg+morph+fMRI 123 subjects. SVC**

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import VotingClassifier
import numpy as np

# Define your custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define three pipelines, each with an SVC classifier
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,75))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(75,1002))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(1002,7672))),
    ("scaler", StandardScaler()),
#     ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2),
    ('df3-clf3', pipe3)
], voting='soft')

# Load your data and target
target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    },
        {
        'df3-clf3__clf__C': [0.01, 0.1, 1],
        'df3-clf3__clf__class_weight': ['balanced', None],
        'df3-clf3__clf__kernel': ['linear', 'rbf'],
        'df3-clf3__clf__probability': [True],
        'df3-clf3__clf__random_state': [seed]
    }
    
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

The mean score using nested cross-validation is: 0.686 ± 0.135
The mean score using nested cross-validation is: 0.724 ± 0.087
The mean score using nested cross-validation is: 0.705 ± 0.101


## **6.4 LF. 75reg+morph+fMRI 123 subjects. XGBoost**

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self


# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,75))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(75,1006))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(1006,7674))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
test_acc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1,  scoring='accuracy')
test_auc = cross_val_score(model, data_late, target, cv=outer_cv,n_jobs=-1, scoring=make_scorer(roc_auc_score))
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_acc.mean():.3f} ± {test_acc.std():.3f}")
print(f"The mean score using nested cross-validation is: "
      f"{test_auc.mean():.3f} ± {test_auc.std():.3f}")

/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib

The mean score using nested cross-validation is: 0.628 ± 0.111
The mean score using nested cross-validation is: 0.644 ± 0.108
The mean score using nested cross-validation is: 0.632 ± 0.107


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib